### Configuration

In [1]:
import os
import numpy as np
import pandas as pd
import cebra
from scipy.stats import zscore
from sklearn.decomposition import PCA

In [2]:
import utils__config
os.chdir(utils__config.working_directory)
os.getcwd()

'g:\\My Drive\\Residency\\Research\\Lab - Damisah\\Project - Sleep'

### Parameters

In [3]:
# hypno_path = 'Cache/Subject01/S01_hypno_epochs_1s.csv' 
# lfp_path = 'Cache/Subject01/S01_sw_epochs_1s.csv'
# spike_path = 'Cache/Subject01/S01_spike_epochs_1s.csv'
# key_prefix = 'S01_Feb02_'

# hypno_path = 'Cache/Subject02/Apr26/S02_hypno_epochs_1s.csv' 
# lfp_path = 'Cache/Subject02/Apr26/S02_sw_epochs_1s.csv'
# spike_path = 'Cache/Subject02/Apr26/S02_spike_epochs_1s.csv'
# key_prefix = 'S02_Apr26_'

hypno_path = 'Cache/Subject02/Apr26/S02_hypno_epochs_1s.csv' 
lfp_path = 'Cache/Subject02/Apr26/S02_sw_epochs_1s.csv'
spike_path = 'Cache/Subject02/Apr26/S02_spike_epochs_1s.csv'
key_prefix = 'S02_Apr27_'

In [4]:
micro_regions = ['CLA']
hdf5_path = 'Cache/cebra_data.h5'

### Reformat Data

In [10]:
# Load Data
hypno = pd.read_csv(hypno_path)
lfp = pd.read_csv(lfp_path)
spikes = pd.read_csv(spike_path)

# Extract zSWA for all channels
lfp = lfp.pivot(index = 'epoch', columns = 'channel', values = 'zlog_power')
lfp.reset_index(drop = True, inplace = True)
lfp.columns.name = None

# PCA of zSWA of all channels, pick top 3 PC's
lfp_pca = PCA(n_components = 3, random_state = 42)
lfp_pca = lfp_pca.fit_transform(lfp)
lfp = pd.DataFrame(data = lfp_pca, columns = ['zSWA_PC1', 'zSWA_PC2', 'zSWA_PC3'])

# Extract zFR for units in the selected regions
spikes = spikes[spikes.unit_region.isin(micro_regions)]
spikes['zfr'] = spikes.groupby(['unit_id'])['fr'].transform(zscore).round(3)
spikes = spikes.pivot(index = 'epoch', columns = 'unit_id', values = 'zfr')
spikes.reset_index(drop = True, inplace = True)
spikes.columns.name = None

# Save pandas DF's to HDF5 for reloading
hypno.to_hdf(hdf5_path, key = key_prefix + 'hypno')
lfp.to_hdf(hdf5_path, key = key_prefix + 'lfp')
spikes.to_hdf(hdf5_path, key = key_prefix + 'spikes')